This section downloads journal papers arxiv, core, jcmns, and lenr_canr

In [ ]:
# Install dependencies (run once)
!pip install beautifulsoup4 requests arxiv --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
# Remove all files and subfolders inside /content/arxiv_lenr_papers
!rm -rf /content/arxiv_lenr_papers/*

# (Optional) If you also want to delete and recreate the folder itself:
!rm -rf /content/arxiv_lenr_papers && mkdir /content/arxiv_lenr_papers

# Remove all files and subfolders inside /content/lenr_canr_papers
!rm -rf /content/lenr_canr_papers/*

# (Optional) If you also want to delete and recreate the folder itself:
!rm -rf /content/lenr_canr_papers && mkdir /content/lenr_canr_papers

In [ ]:
#This code downloads all of iccf conference proceeedings from https://lenr-canr.org/wordpress/?page_id=2130 and  lenr papers from  arxiv
import os
import requests
from bs4 import BeautifulSoup
import arxiv

# ── Part 1: Scrape and download PDFs from the LENR‑CANR site ──────────────────────
def download_lenr_canr_papers(output_folder='lenr_canr_papers'):
    url = "https://lenr-canr.org/wordpress/?page_id=2130"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    pdf_links = [
        a['href']
        for a in soup.find_all('a', href=True)
        if a['href'].lower().endswith('.pdf')
    ]
    os.makedirs(output_folder, exist_ok=True)
    for link in pdf_links:
        fname = os.path.basename(link.split('?')[0])
        out_path = os.path.join(output_folder, fname)
        if not os.path.exists(out_path):
            print(f"Downloading {fname}...")
            r = requests.get(link)
            with open(out_path, 'wb') as f:
                f.write(r.content)
    print(f"Finished downloading {len(pdf_links)} papers from LENR‑CANR into ./{output_folder}/")

# ── Part 2: Query and download LENR‑specific papers from arXiv ──────────────────
def download_arxiv_lenr(output_folder='arxiv_lenr_papers', max_results=200):
    # Limit search to titles for high relevance
    query = 'ti:"cold fusion" OR ti:"LENR" OR ti:"low energy nuclear reaction"'
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    os.makedirs(output_folder, exist_ok=True)
    count = 0
    for result in search.results():
        title = result.title.lower()
        summary = result.summary.lower()
        # Double-check keywords in title or abstract
        if any(kw in title or kw in summary for kw in ["cold fusion", "lenr", "low energy nuclear"]):
            paper_id = result.entry_id.split('/')[-1]
            filename = f"{paper_id}.pdf"
            path = os.path.join(output_folder, filename)
            if not os.path.exists(path):
                print(f"Downloading {filename}: {result.title}")
                r = requests.get(result.pdf_url)
                with open(path, 'wb') as f:
                    f.write(r.content)
                count += 1
    print(f"Finished downloading {count} LENR‑related arXiv papers into ./{output_folder}/")

# Run both parts
download_lenr_canr_papers()
download_arxiv_lenr()


Finished downloading 73 papers from LENR‑CANR into ./lenr_canr_papers/


/tmp/ipython-input-3501317175.py:39: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Finished downloading 29 LENR‑related arXiv papers into ./arxiv_lenr_papers/


In [ ]:
#this code  searches and downloads all articles on https://jcmns.org/articles

# 1) Install & import
!pip install --quiet requests tqdm

import os
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 2) Configuration
BASE    = "https://jcmns.org/article"
OUT_DIR = "/content/jcmns_papers"
os.makedirs(OUT_DIR, exist_ok=True)

# 3) Helper to check & download one PDF
def fetch_pdf(article_id):
    url = f"{BASE}/{article_id}.pdf"
    # first do a HEAD to avoid downloading 404 pages
    h = requests.head(url)
    if h.status_code != 200 or "pdf" not in h.headers.get("Content-Type",""):
        return None
    # now GET and stream‐write
    out_path = os.path.join(OUT_DIR, f"{article_id}.pdf")
    if os.path.exists(out_path):
        return article_id  # already got it
    r = requests.get(url, stream=True)
    if r.status_code == 200 and "pdf" in r.headers.get("Content-Type",""):
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
        return article_id
    return None

# 4) Scan a range of IDs in parallel
#    JCMNS IDs seem to run roughly from ~72000 up into the 130000s.
START_ID = 70000
END_ID   = 135000
MAX_WORKERS = 10

found = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
    futures = { pool.submit(fetch_pdf, i): i for i in range(START_ID, END_ID+1) }
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Scanning IDs"):
        result = fut.result()
        if result is not None:
            print(f"[✔] Downloaded article/{result}.pdf")
            found.append(result)

print(f"\n✅ Done! Retrieved {len(found)} PDFs into {OUT_DIR}")


Scanning IDs:  85%|████████▍ | 54995/65001 [10:19<02:02, 81.75it/s]

[✔] Downloaded article/124957.pdf


Scanning IDs: 100%|██████████| 65001/65001 [12:13<00:00, 88.60it/s] 


✅ Done! Retrieved 1 PDFs into /content/jcmns_papers


In [ ]:
"""
Downloads PDFs for works returned by the CORE API v3 search. Uses resilient HTTP GET with retry for 429/500, randomized pacing to avoid rate limits,
plus multiple fallbacks: landing‑page scrape, metadata URLs, CORE display‑page scrape, CORE download API, and Unpaywall.
"""
# 1) Install dependencies
#    pip install requests tqdm beautifulsoup4

import os, time, random, requests
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup

# 2) Configuration
API_KEY_CORE    = "WbqHTSlGpr6nyZkEuIQtjU0s1hY7zafC"  # replace with your CORE API key
UNPAYWALL_EMAIL = "youremail@example.com"          # replace with your email for Unpaywall
QUERY           = "low energy nuclear reaction OR cold fusion"
PAGE_SIZE       = 100
MAX_PAGES       = 10
MAX_WORKERS     = 8
OUT_DIR         = "core_papers"
CORE_API        = "https://api.core.ac.uk/v3"
CORE_DOMAIN     = "https://core.ac.uk"
HAS_FULLTEXT    = True  # restrict to works with full text metadata

os.makedirs(OUT_DIR, exist_ok=True)

# 3) Resilient HTTP GET with retry on 429/500

def http_get(url, max_retries=None, **kwargs):
    backoff = 5
    attempts = 0
    while True:
        attempts += 1
        try:
            r = requests.get(url, **kwargs)
        except Exception as e:
            print(f"[http-error] {url}: {e}. retrying in {backoff}s...")
            time.sleep(backoff)
            backoff = min(backoff*2, 300)
            continue
        if r.status_code == 200:
            return r
        if r.status_code in (429, 500):
            ra = r.headers.get('Retry-After')
            wait = int(ra) if ra and ra.isdigit() else backoff
            if max_retries and attempts >= max_retries:
                return r
            print(f"[http {r.status_code}] {url}. retrying in {wait}s...")
            time.sleep(wait)
            backoff = min(backoff*2, 300)
            continue
        return r

# 4) Fetch a page of CORE works

def fetch_page(page):
    url = f"{CORE_API}/search/works"
    headers = {"Authorization": API_KEY_CORE}
    params = {"q": QUERY, "page": page, "pageSize": PAGE_SIZE}
    if HAS_FULLTEXT:
        params['hasFullText'] = True
    r = http_get(url, headers=headers, params=params, timeout=30)
    if r.status_code == 200:
        return r.json().get('results', [])
    print(f"[page {page}] HTTP {r.status_code}. skipping")
    return []

# 5) Download PDF for a work with multiple fallbacks

def download_work(work):
    time.sleep(random.uniform(0.5, 2.0))  # pace requests
    work_id = work.get('id', 'no-id')
    doi     = work.get('doi')
    landing = work.get('url')
    fname   = f"{work_id}.pdf"
    outpath = os.path.join(OUT_DIR, fname)
    if os.path.exists(outpath):
        print(f"[exists] {fname}")
        return

    def save_pdf(resp, tag):
        with open(outpath, 'wb') as f:
            for chunk in resp.iter_content(8192):
                f.write(chunk)
        print(f"[{tag}] {fname}")
        return True

    # A) Landing page scrape
    if landing:
        r0 = http_get(landing, timeout=20)
        if r0.status_code == 200:
            soup0 = BeautifulSoup(r0.text, 'html.parser')
            candidates = soup0.find_all(['a','iframe'], href=lambda u: u and u.lower().endswith('.pdf'))
            meta = soup0.find('meta', attrs={'name':'citation_pdf_url'})
            if meta:
                candidates.append(meta)
            for tag in candidates:
                pdf0 = tag.get('href') or tag.get('src') or tag.get('content')
                if not pdf0: continue
                pdf0 = pdf0 if pdf0.startswith('http') else urljoin(CORE_DOMAIN, pdf0)
                r0b = http_get(pdf0, stream=True, timeout=40)
                ct0 = r0b.headers.get('Content-Type','').lower()
                if r0b.status_code==200 and 'pdf' in ct0:
                    if save_pdf(r0b,'landing-scrape'): return
            print(f"[landing-skip] {landing}")

    # B) Metadata URL fields
    urls = []
    for key in ('downloadUrl','downloadUrls','fullTextIdentifier','openAccessPdf'):
        v = work.get(key)
        if isinstance(v,str): urls.append(v)
        elif isinstance(v,list): urls.extend(v)
        elif isinstance(v,dict) and v.get('url'): urls.append(v['url'])
    # arXiv fallback
    for u in list(urls):
        if 'arxiv.org/abs/' in u:
            aid = u.rsplit('/',1)[-1]
            urls.append(f"https://arxiv.org/pdf/{aid}.pdf")

    for url in set(urls):
        if not url.startswith('http'): continue
        r2 = http_get(url, stream=True, timeout=60, max_retries=2)
        ct2 = r2.headers.get('Content-Type','').lower()
        if r2.status_code==200 and ('application/pdf' in ct2 or r2.url.lower().endswith('.pdf')):
            if save_pdf(r2,'meta-url'): return
        print(f"[url-skip] {url} CT={ct2}")

    # C) CORE display‑page scrape
    disp = f"{CORE_DOMAIN}/display/{work_id}"
    r3 = http_get(disp, timeout=20)
    if r3.status_code==200:
        soup1 = BeautifulSoup(r3.text,'html.parser')
        link1 = soup1.find('a', href=lambda u: u and u.lower().endswith('.pdf'))
        if link1:
            pdf1 = link1['href']
            pdf1 = pdf1 if pdf1.startswith('http') else urljoin(CORE_DOMAIN,pdf1)
            r3b = http_get(pdf1, stream=True, timeout=40)
            ct3 = r3b.headers.get('Content-Type','').lower()
            if r3b.status_code==200 and 'pdf' in ct3:
                if save_pdf(r3b,'display-scrape'): return
        print(f"[display-skip] {disp}")

    # D) CORE download API
    dl = f"{CORE_API}/works/{work_id}/download?mediaType=application/pdf"
    r4 = http_get(dl, headers={"Authorization":API_KEY_CORE}, stream=True, timeout=60, max_retries=2)
    ct4 = r4.headers.get('Content-Type','').lower()
    if r4.status_code==200 and ('application/pdf' in ct4 or r4.url.lower().endswith('.pdf')):
        if save_pdf(r4,'download-core'): return
    print(f"[core-skip] {work_id} CT={ct4}")

    # E) Unpaywall fallback
    if doi:
        up = http_get(f"https://api.unpaywall.org/v2/{doi}?email={UNPAYWALL_EMAIL}", timeout=30)
        if up.status_code==200:
            data = up.json()
            loc = data.get('best_oa_location') or (data.get('oa_locations') or [None])[0]
            pdf = loc.get('url_for_pdf') if isinstance(loc,dict) else None
            if pdf:
                r5 = http_get(pdf, stream=True, timeout=60)
                ct5 = r5.headers.get('Content-Type','').lower()
                if r5.status_code==200 and 'application/pdf' in ct5:
                    if save_pdf(r5,'unp-url'): return
                print(f"[unp-skip] {pdf} CT={ct5}")

    print(f"[fail] {work_id}, no PDF found")

# 6) Main
all_works=[]
for p in range(1,MAX_PAGES+1):
    time.sleep(random.uniform(1,2))
    ws = fetch_page(p)
    print(f"➤ Page {p}: {len(ws)} works")
    all_works.extend(ws)
# filter candidates
all_works = [w for w in all_works if w.get('url') or w.get('downloadUrl') or w.get('doi')]
print(f"\n🎯 Candidate works: {len(all_works)} of {PAGE_SIZE*MAX_PAGES}\n")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    list(ex.map(download_work, all_works))

print(f"\n✅ Done. PDFs in '{OUT_DIR}'")


[http 500] https://api.core.ac.uk/v3/search/works. retrying in 5s...
➤ Page 1: 4 works
➤ Page 2: 4 works
➤ Page 3: 4 works
➤ Page 4: 4 works
➤ Page 5: 4 works
➤ Page 6: 4 works
[http 500] https://api.core.ac.uk/v3/search/works. retrying in 5s...
[http 500] https://api.core.ac.uk/v3/search/works. retrying in 10s...
[http 500] https://api.core.ac.uk/v3/search/works. retrying in 20s...
➤ Page 7: 4 works
➤ Page 8: 4 works
➤ Page 9: 4 works
➤ Page 10: 4 works

🎯 Candidate works: 40 of 1000

[url-skip] http://arxiv.org/abs/nucl-th/0303057 CT=text/html; charset=utf-8
[url-skip] https://arxiv.org/pdf/0303057.pdf CT=text/html; charset=utf-8
[url-skip] https://core.ac.uk/download/161936859.pdf CT=text/html; charset=utf-8
[url-skip] https://core.ac.uk/download/161936859.pdf CT=text/html; charset=utf-8
[url-skip] https://core.ac.uk/download/44139449.pdf CT=text/html; charset=utf-8
[url-skip] https://core.ac.uk/download/44139449.pdf CT=text/html; charset=utf-8
[url-skip] http://arxiv.org/abs/nucl-t

This  section fits a topic  model  to the lenr  corpus

In [ ]:
# Colab-specific: install dependencies and spaCy model
!pip install bertopic sentence-transformers pdfminer.six fuzzywuzzy[speedup] spacy -quiet
!python -m spacy download en_core_web_sm


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install pdfminer.six  --quiet bertopic sentence-transformers pdfminer.six fuzzywuzzy[speedup] spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
"""
Topic Modeling and Concept Extraction Pipeline for LENR Papers

1) Extract text from PDFs in given directories.
2) Fit BERTopic to discover ~10-20 coherent topics.
3) Collect top N terms per topic and canonicalize variants via fuzzy matching.
4) Use canonical term list to seed a spaCy EntityRuler for NER.
"""
# Colab-specific: install dependencies and spaCy model
#!pip install --quiet bertopic sentence-transformers pdfminer.six fuzzywuzzy[speedup] spacy
#!python -m spacy download en_core_web_sm

import os, logging
from pathlib import Path
from pdfminer.high_level import extract_text
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from fuzzywuzzy import process
import spacy
from spacy.pipeline import EntityRuler

# Suppress PDFMiner color warnings
logging.getLogger("pdfminer.pdfinterp").setLevel(logging.ERROR)

# 1) Load and extract text from PDFs
PDF_DIRS = [
    "/content/arxiv_lenr_papers",
    "/content/core_papers",
    "/content/jcmns_papers",
    "/content/lenr_canr_papers"
]

corpus = []
ids = []
for d in PDF_DIRS:
    for pdf in Path(d).glob("*.pdf"):
        try:
            text = extract_text(str(pdf))
            if text and len(text) > 100:
                corpus.append(text)
                ids.append(pdf.stem)
            else:
                print(f"[skip] {pdf.name}: empty or too short")
        except Exception as e:
            print(f"[skip] {pdf.name}: {e}")

# 2) Fit BERTopic
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words="english")
topic_model = BERTopic(nr_topics=15, vectorizer_model=vectorizer)
topics, probs = topic_model.fit_transform(corpus)

# Show topic info
info = topic_model.get_topic_info()
print(info)

# 3) Collect and canonicalize top terms
term_lists = []
for t in info.Topic.iloc[1:]:  # skip -1 outliers
    terms = [word for word, _ in topic_model.get_topic(int(t))[:10]]
    term_lists.extend(terms)

canonical = {}
for term in sorted(set(term_lists)):
    if canonical:
        best = process.extractOne(term, list(canonical.keys()))
        if best is not None and len(best) == 2 and best[1] >= 85:
            match = best[0]
            canonical[match].add(term)
            continue
    canonical[term] = {term}

# Build canonical term groups
canonical_terms = {key: variants for key, variants in canonical.items()}
print("Canonical term groups:")
for key, group in canonical_terms.items():
    print(f"{key}: {group}")

# 4) Seed spaCy EntityRuler
nlp = spacy.load("en_core_web_sm")
# add entity ruler by factory name
ruler = nlp.add_pipe("entity_ruler", config={"overwrite_ents": True}, first=True)

patterns = []
for canonical_key, variants in canonical_terms.items():
    patterns.append({"label": "LENR_CONCEPT", "pattern": canonical_key})
    for variant in variants:
        if variant != canonical_key:
            patterns.append({"label": "LENR_CONCEPT", "pattern": variant})
ruler.add_patterns(patterns)

# Example usage: extract entities from first doc
if corpus:
    doc = nlp(corpus[0])
    print([ent.text for ent in doc.ents if ent.label_ == "LENR_CONCEPT"])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   Topic  Count                          Name  \
0     -1      7  -1_fusion_energy_excess_heat   
1      0     45    0_fusion_pd_energy_nuclear   
2      1     27       1_energy_fusion_heat_pd   
3      2     24  2_energy_nucleus_nuclei_phys   

                                      Representation  \
0  [fusion, energy, excess, heat, cold, nuclear, ...   
1  [fusion, pd, energy, nuclear, heat, neutron, d...   
2  [energy, fusion, heat, pd, excess, nuclear, ce...   
3  [energy, nucleus, nuclei, phys, 10, nuclear, p...   

                                 Representative_Docs  
0  [EPRI. Proceedings: Fourth International Confe...  
1  [Celani, F., et al. Thermal and Isotopic Anoma...  
2  [This document is: \n\nPons, S., ed. Proceedin...  
3  [Submitted to Journal of Condensed Matter Nucl...  
Canonical term groups:
10: {'10'}
barrier: {'barrier'}
cell: {'cell'}
cold: {'cold'}
deuterium: {'deuterium'}
electrolysis: {'electrolysis'}
energy: {'energy'}
excess: {'excess'}
experiments: {'expe

In [ ]:
"""
Topic Modeling and Concept Extraction Pipeline for LENR Papers

1) Extract text from PDFs in given directories.
2) Fit BERTopic to discover ~10–20 coherent topics.
3) Collect top N terms per topic and canonicalize variants via fuzzy matching.
4) Use canonical term list to seed a spaCy EntityRuler for NER.
"""
# Colab-specific: install dependencies and spaCy model
#!pip install --quiet bertopic sentence-transformers pdfminer.six fuzzywuzzy[speedup] spacy
#!python -m spacy download en_core_web_sm

import os, logging
from pathlib import Path
from pdfminer.high_level import extract_text
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from fuzzywuzzy import process
import spacy
from spacy.pipeline import EntityRuler

# Suppress PDFMiner color warnings
logging.getLogger("pdfminer.pdfinterp").setLevel(logging.ERROR)

# 1) Load and extract text from PDFs
PDF_DIRS = [
    "/content/arxiv_lenr_papers",
    "/content/core_papers",
    "/content/jcmns_papers",
    "/content/lenr_canr_papers"
]

corpus = []
ids = []
for d in PDF_DIRS:
    for pdf in Path(d).glob("*.pdf"):
        try:
            text = extract_text(str(pdf))
            if text and len(text) > 100:
                corpus.append(text)
                ids.append(pdf.stem)
            else:
                print(f"[skip] {pdf.name}: empty or too short")
        except Exception as e:
            print(f"[skip] {pdf.name}: {e}")

# 2) Fit BERTopic
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words="english")
topic_model = BERTopic(nr_topics=15, vectorizer_model=vectorizer)
topics, probs = topic_model.fit_transform(corpus)

# Show topic info
info = topic_model.get_topic_info()
print(info)

# 3) Collect and canonicalize top terms
term_lists = []
for t in info.Topic.iloc[1:]:  # skip -1 outliers
    terms = [word for word, _ in topic_model.get_topic(int(t))[:10]]
    term_lists.extend(terms)

canonical = {}
for term in sorted(set(term_lists)):
    if canonical:
        best = process.extractOne(term, list(canonical.keys()))
        if best and best[1] >= 85:
            canonical[best[0]].add(term)
            continue
    canonical[term] = {term}

# Filter out numeric-only or very short keys
droplist = [k for k in canonical if k.isdigit() or len(k) < 3]
for k in droplist:
    canonical.pop(k, None)

# Build canonical term groups
canonical_terms = canonical
print("Canonical term groups:")
for key, group in canonical_terms.items():
    print(f"{key}: {group}")

# 4) Seed spaCy EntityRuler
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler", config={"overwrite_ents": True}, first=True)

patterns = []
for canonical_key, variants in canonical_terms.items():
    patterns.append({"label": "LENR_CONCEPT", "pattern": canonical_key})
    for variant in variants:
        if variant != canonical_key:
            patterns.append({"label": "LENR_CONCEPT", "pattern": variant})
ruler.add_patterns(patterns)

# Example usage: extract unique entities from first doc
if corpus:
    doc = nlp(corpus[0])
    ents = {ent.text for ent in doc.ents if ent.label_ == "LENR_CONCEPT"}
    print(sorted(ents))


   Topic  Count                          Name  \
0     -1      7  -1_fusion_energy_excess_cold   
1      0     46    0_fusion_pd_energy_nuclear   
2      1     26   1_energy_fusion_heat_excess   
3      2     24  2_energy_nucleus_nuclei_phys   

                                      Representation  \
0  [fusion, energy, excess, cold, heat, cold fusi...   
1  [fusion, pd, energy, nuclear, neutron, heat, d...   
2  [energy, fusion, heat, excess, pd, nuclear, ce...   
3  [energy, nucleus, nuclei, phys, 10, nuclear, b...   

                                 Representative_Docs  
0  [The Sixth  International Conf ere nee on  Co...  
1  [Celani, F., et al. Thermal and Isotopic Anoma...  
2  [This document is: \n\nPons, S., ed. Proceedin...  
3  [Submitted to Journal of Condensed Matter Nucl...  
Canonical term groups:
barrier: {'barrier'}
cell: {'cell'}
deuterium: {'deuterium'}
electrolysis: {'electrolysis'}
energy: {'energy'}
excess: {'excess'}
experiments: {'experiments'}
fusion: {'fusion

In [ ]:
!pip install stanford-openie

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.7 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=3d2d1df38ef0ce3cd05546215a4116b499ccd73b57df1b330fed07288f49b803
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery-connection 1.18.3 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-

In [ ]:
!pip install --quiet langchain-ollama python-docx nest_asyncio tqdm

import os, threading, subprocess, time, requests

# Unset any OpenAI fallbacks
for v in ["OPENAI_API_KEY","LITELLM_PROVIDER","LITELL M_MODEL","LITELL M_BASE_URL"]:
    os.environ.pop(v, None)
os.environ["OLLAMA_HOST"]    = "0.0.0.0:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

!curl -fsSL https://ollama.com/install.sh -o install.sh
!bash install.sh && chmod +x /usr/local/bin/ollama

def _serve_ollama():
    subprocess.Popen(["ollama","serve"], stderr=subprocess.DEVNULL)
threading.Thread(target=_serve_ollama, daemon=True).start()
time.sleep(8)
print("✅ Ollama status:", requests.get("http://127.0.0.1:11434").status_code)

!ollama pull deepseek-r1:1.5b

from langchain_ollama import OllamaLLM

OLLAMA_MODEL = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://127.0.0.1:11434", temperature=0.7)

In [ ]:
!pip install --quiet bertopic sentence-transformers pdfminer.six fuzzywuzzy[speedup] spacy requests
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
#  0) Colab setup: install dependencies & spaCy model
# ─────────────────────────────────────────────────────────────────────────────
#!pip install --quiet \
#    langchain-ollama python-docx nest_asyncio tqdm \
#    bertopic sentence-transformers pdfminer.six fuzzywuzzy[speedup] \
#    spacy requests

#!python -m spacy download en_core_web_sm

# ─────────────────────────────────────────────────────────────────────────────
#  1) Start Ollama server in the background and pull a free model
# ─────────────────────────────────────────────────────────────────────────────
import os, threading, subprocess, time, requests

# unset any OpenAI fallbacks
for v in ["OPENAI_API_KEY","LITELLM_PROVIDER","LITELL M_MODEL","LITELL M_BASE_URL"]:
    os.environ.pop(v, None)

# point at our local Ollama
os.environ["OLLAMA_HOST"]    = "127.0.0.1:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

# install & serve Ollama
!curl -fsSL https://ollama.com/install.sh -o install.sh
!bash install.sh && chmod +x /usr/local/bin/ollama

def _serve_ollama():
    subprocess.Popen(["ollama","serve"], stderr=subprocess.DEVNULL)
threading.Thread(target=_serve_ollama, daemon=True).start()
time.sleep(8)
print("✅ Ollama status:", requests.get("http://127.0.0.1:11434").status_code)

# pull a model
!ollama pull deepseek-r1:1.5b

# ─────────────────────────────────────────────────────────────────────────────
#  2) Imports & logging
# ─────────────────────────────────────────────────────────────────────────────
import os, logging, json, re
from pathlib import Path
from pdfminer.high_level import extract_text
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from fuzzywuzzy import process
import spacy
from spacy.pipeline import EntityRuler
import requests

# suppress PDFMiner warnings
logging.getLogger("pdfminer.pdfinterp").setLevel(logging.ERROR)

# ─────────────────────────────────────────────────────────────────────────────
#  3) Step 1: extract text from all PDFs
# ─────────────────────────────────────────────────────────────────────────────
PDF_DIRS = [
    "/content/arxiv_lenr_papers",
    "/content/core_papers",
    "/content/jcmns_papers",
    "/content/lenr_canr_papers"
]

corpus, ids = [], []
for d in PDF_DIRS:
    for pdf in Path(d).glob("*.pdf"):
        try:
            txt = extract_text(str(pdf))
            if txt and len(txt) > 200:
                corpus.append(txt)
                ids.append(pdf.stem)
            else:
                print(f"[skip] too short: {pdf.name}")
        except Exception as e:
            print(f"[skip] {pdf.name}: {e}")

print(f"\n✅ Loaded {len(corpus)} documents.\n")

# ─────────────────────────────────────────────────────────────────────────────
#  4) Step 2: fit BERTopic for ~15 topics
# ─────────────────────────────────────────────────────────────────────────────
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words="english")
topic_model = BERTopic(nr_topics=15, vectorizer_model=vectorizer)
topics, probs = topic_model.fit_transform(corpus)

info = topic_model.get_topic_info()
print(info)

# ─────────────────────────────────────────────────────────────────────────────
#  5) Step 3: canonicalize top terms via fuzzy matching
# ─────────────────────────────────────────────────────────────────────────────
term_lists = []
for t in info.Topic.iloc[1:]:  # skip -1 = outliers
    for word, _ in topic_model.get_topic(int(t))[:10]:
        term_lists.append(word)

canonical = {}
for term in sorted(set(term_lists)):
    if canonical:
        best = process.extractOne(term, list(canonical.keys()))
        if best and best[1] >= 85:
            canonical[best[0]].add(term)
            continue
    canonical[term] = {term}

# drop numeric‑only or too‑short
for k in list(canonical):
    if k.isdigit() or len(k) < 3:
        canonical.pop(k)

print("\nCanonical term groups:")
for k,v in canonical.items():
    print(f"  {k}: {v}")

# ─────────────────────────────────────────────────────────────────────────────
#  6) Step 4: seed spaCy EntityRuler
# ─────────────────────────────────────────────────────────────────────────────
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler", config={"overwrite_ents":True}, first=True)

patterns = []
for key, variants in canonical.items():
    patterns.append({"label":"LENR_CONCEPT", "pattern":key})
    for var in variants:
        if var != key:
            patterns.append({"label":"LENR_CONCEPT", "pattern":var})
ruler.add_patterns(patterns)

# sample
if corpus:
    doc = nlp(corpus[0])
    ents = {ent.text for ent in doc.ents if ent.label_=="LENR_CONCEPT"}
    print("\nSample concepts in doc0:", ents)

# ─────────────────────────────────────────────────────────────────────────────
#  7) Step 5: relation extraction via Ollama
# ─────────────────────────────────────────────────────────────────────────────
OLLAMA_API  = "http://127.0.0.1:11434/v1/completions"
MODEL_NAME  = "deepseek-r1:1.5b"
HEADERS     = {"Content-Type":"application/json"}

def extract_relations_ollama(sentence, concepts):
    prompt = (
        "Extract a JSON array of relation triples "
        "{subject, relation, object} where subject and object are among "
        f"{concepts}.\n\nSentence: \"{sentence}\""
    )
    payload = {
        "model":       MODEL_NAME,
        "prompt":      prompt,
        "max_tokens":  256,
        "temperature": 0.0
    }
    r = requests.post(OLLAMA_API, headers=HEADERS, json=payload)
    r.raise_for_status()
    resp = r.json()
    text = resp["choices"][0]["text"].strip()

    m = re.search(r"(\[.*\])", text, flags=re.DOTALL)
    if not m:
        return []

    try:
        arr = json.loads(m.group(1))
    except json.JSONDecodeError:
        return []

    # filter out any non‐dict items
    return [item for item in arr if isinstance(item, dict)]


# loop over sentences
triples = []
for doc_text in corpus:
    doc = nlp(doc_text)
    for sent in doc.sents:
        ents = [e.text for e in sent.ents if e.label_=="LENR_CONCEPT"]
        if len(ents) < 2:
            continue

        rels = extract_relations_ollama(sent.text, ents)
        for rel in rels:
            if not isinstance(rel, dict):
                # skip anything that isn’t a dict
                continue

            s = rel.get("subject")
            r = rel.get("relation")
            o = rel.get("object")

            # only keep well-formed triples over your canonical terms
            if s in canonical and o in canonical:
                triples.append((s, r, o))


print(f"\n✅ Extracted {len(triples)} LENR triples via Ollama.\n")

# save
with open("lenr_triples.json","w") as f:
    json.dump(triples,f,indent=2)
print("Saved ▶ lenr_triples.json")


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
✅ Ollama status: 200


✅ Loaded 103 documents.

   Topic  Count                             Name  \
0     -1      2  -1_heat_cell_excess_temperature   
1      0     46       0_fusion_pd_energy_nuclear   
2      1     25     1_energy_nucleus_nuclei_phys   
3      2     30      2_energy_fusion_heat_excess   

                                      Representation  \
0  [heat, cell, excess, temperature, excess heat,...   
1  [fusion, pd, energy, nuclear, neutron, heat, d...   
2  [energy, nucleus, nuclei, phys, nuclear, 10, p...   
3  [energy, fusion, heat, excess, nuclear, pd, co..